In [1]:
from pathlib import Path
import pandas as pd
import json
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm 

from pairing_utils import get_pairing
from markup_data import Manga

In [2]:
root = Path('../../../data/NER/')
raw_root = root / 'raw'
processed_root = root / 'processed'

In [15]:
df = pd.read_csv(processed_root / 'merged_grouple_data.csv')
print('all', df.shape)
print('with comments', df[~df.volumes_data_file.isna()].shape)
df = df[~df.volumes_data_file.isna()]
df.head()

all (8142, 3)
with comments (1670, 3)


,url,page_file,volumes_data_file
0,naruto_dodzinsi__novogodnii_festival,0.json,1484.json
1,naruto_dj___icha_icha_academy,1.json,712.json
2,naruto_dj____nikki,10.json,852.json
3,azur_lane___cl_ust_01,100.json,693.json
4,fate_zero_dj___falling_down,1000.json,1285.json


# Load comments for each manga

In [4]:
def load_data(page_file, volumes_file):
    page = raw_root / 'grouple_manga_pages/' / page_file
    volumes = raw_root / 'grouple_merged_volumes/' / volumes_file
    return {'page': json.load(open(page)),
            'volumes': json.load(open(volumes))}

In [5]:
dataset = []
N = 100

# def hotfix(pname):
#     for i, c in enumerate(pname[1:], start=1):
#         if c.isupper():
#             return pname[:i]
#     return pname
        
for name, page, volumes in tqdm(df.to_records(index=False)[:N]):
    data = load_data(page, volumes)
    url = data['page']['url']
    descr = data['page']['description']
    comments = data['volumes']['comments']
    manga = Manga(comments)
    dataset.append(manga)

  0%|          | 0/100 [00:00<?, ?it/s]

In [18]:
comms = dataset[0].comments.comments
comms[:10]

['Мне эта додзя всегда так сильно нравилась',
 'Ни ч так   Искренне',
 'Мило   Коротко и романтично',
 'Ннн Это не яой это больше сненай Даже поцелуйчика не было  разочарована',
 'Женя как всегда не увидела в этом никакого смысла',
 'Это как немое кино',
 'Мне очень понравилось',
 'ПЕрвый страницы думала  манга для тех кто не умеет читать чтоли Оо а вот потом',
 'так мило только вот вначеле перевод ещ бы сделали а то на немой фильм смахива',
 'спасибо за перевод милая додзя']

In [13]:
import random
from typing import List

class Saver:
    """Class for sampling and saving labled comments data"""
    def __init__(self, save_path: str, dataset: list):
        self.path = save_path
        self.dataset = dataset
        self.used = set()
    
    def sample(self) -> List[str, ]:
        # Fetches sample from data
        def get_index():
            m = random.randint(0, len(self.dataset)-1)
            c = random.randint(0, len(self.dataset[m].comments.comments)-1)
            return m, c
        
        m, c = get_index()
        idx = f'{m}_{c}'
        while idx in self.used:
            m, c = get_index()
            idx = f'{m}_{c}'
            
        self.used.add(idx)
        comm = self.dataset[m].comments.comments[c]
        return comm.split()
        
    def add(self, words: List[str, ], labels: List[str, ]) -> None:
        # Writes out labled sample
        with open(self.path, 'a') as f:
            for w, l in zip(words, labels):
                f.write(f'{w} _ _ {l}\n')
            f.write('\n')
            
saver = Saver(processed_root / 'comments/raw.txt', dataset)

In [11]:
# Manually label i-th word(s) as PER(son)

while 1:
    words = saver.sample()
    labels = ['O']*len(words)
    s = [(i,w) for i,w in enumerate(words)]
    s = '\n'.join(list(map(lambda p: '%d %s' % p, s)))
    print(s)
    ixs = input()
    if len(ixs) > 0:
        ixs = list(map(int, ixs.split()))
        for ix in ixs:
            labels[ix] = 'PER'
    saver.add(words, labels)
    print('-----------------')

0 ибо
1 ты
2 на
3 земле
4 божьей
5 приклонись
7 фф


ValueError: invalid literal for int() with base 10: 'фф'

In [12]:
len(open(saver.path).read().split('\n\n'))

FileNotFoundError: [Errno 2] No such file or directory: '..\\..\\..\\data\\NER\\processed\\comments\\raw.txt'